# Custom wind speed and sig profiles

This example shows how you can create your own functions for the wind speed and turbulence standard deviation as a function of spatial location. For simplicity, we will keep the default Kaimal spectrum, but you could also have the spectrum be a custom function as well.

The spatial variation of the mean wind speed and turbulence standard deviation we choose in this example will be to mimic a full-wake scenario. Note that these variables are not necessarily physically realistic, this is just a demonstration!

This example has the following sections:  

* [Preliminaries](#Preliminaries:-importing-functions)  

* [Defining grid and time step](#Define-turbulence-grid-and-time)  

* [Custom wind speed profile](#Custom-wind-speed-profile)  

* [Custom turbulence std dev](#Custom-turbulence-standard-deviation-profile)  

* [Generate and inspect turbulence](#Generate-and-inspect-turbulence)  

**Note**: it is also possible to have a custom power spectrum profile function, but that is not explicitly demonstrated here. However, the method is extremely similar to what is shown here, and more information can be found in the related tab under the Reference Guide.

## Preliminaries: importing functions

Before we can begin, we need to importe a few functions and define some variables that we will need later.

In [ ]:
%matplotlib inline
#import os

import matplotlib.pyplot as plt  # matplotlib for some plotting
import numpy as np  # numeric python functions

from pyconturb import gen_turb, gen_spat_grid  # generate turbulence, rect. grid
from pyconturb.wind_profiles import constant_profile  # useful for freestream

cntr_pos, rad, u_inf = [0, 119], 90, 12  # center pos of grid, radius of "wake", and inflow wsp

## Define turbulence grid and time

Our first step is to define 1) the simulation time/time step and 2) the points of the grid and which turbulence components (i.e., $u$, $v$ and/or $w$) we want to simulate. For this example, we'll pick a fairly dense grid, but we'll only simulate the longitudinal component.

In [ ]:
T, dt = 300, 1  # only simulate 300 seconds for now
y, z = np.linspace(-rad, rad, 15), np.linspace(cntr_pos[1] - rad, cntr_pos[1] + rad, 15)
spat_df = gen_spat_grid(y, z, comps=[0])  # generate grid with only u
print(spat_df.head())  # show a few columns of the spatial dataframe
print('No. of points to simulate: ', spat_df.shape[0])  # number of points to simulate
plt.scatter(spat_df.loc['y'], spat_df.loc['z'])  # show the grid
plt.axis('equal');

## Custom wind speed profile

Now let's define our custom functions for the spatial variation of the mean wind speed. Note that the wind speed function must be of the form
```
wsp_values = wsp_func(y, z, **kwargs)
```
where `kwargs` is a dictionary of the keyword arguments for the profile function. (It can also include unused keyword arguments.)

In [ ]:
def wake_wsp(y, z, cntr_pos=[0, 90], rad=50, u_inf=10, max_def=0.5, **kwargs):
    """Non-realistic wake deficit.
    rad is the wake of the wake, u_inf is undisturbed inflow, and max_def is the max. deficit."""
    dist_to_cntr = np.sqrt((y - cntr_pos[0])**2 + (z - cntr_pos[1])**2)  # distance to center point
    freestream = constant_profile(y, z, u_ref=u_inf)  # no wake deficit
    wake_def = max_def * np.sin(np.pi/2 * (rad - dist_to_cntr) / rad)  # sinusoidal
    wake_def = wake_def * (dist_to_cntr < rad)  # deficit is not outside of rad
    return np.array(freestream - wake_def)  # must return array regardless of input

We can plot a few different waked profiles to check that this function works.

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(9.9, 8))
plot_vals = [([0, 119], 90, 0.5), ([0, 119], 30, 0.5), ([-30, 80], 90, 0.5), ([60, 130], 90, 2)]
for iax, (cnt, r, mdef) in enumerate(plot_vals):
    ax = axs[iax//2, iax%2]
    wsp_values = wake_wsp(spat_df.loc['y'], spat_df.loc['z'], cntr_pos=cnt, rad=r, u_inf=u_inf, max_def=mdef)
    cnt = ax.contourf(y, z, wsp_values.reshape(y.size, z.size).T)
    ax.axis('equal')
    plt.colorbar(cnt, ax=ax);
    ax.set_title(f'{rad}-m rad at {cntr_pos}, max_def={mdef}')

## Custom turbulence standard deviation profile

Just like we did for the mean wind speed, now let us define a function for the turbulence standard deviation as a function of spatial location (and turbulence component). Note that the sig function must be of the form
```
sig_values = sig_func(k, y, z, **kwargs)
```
where `kwargs` is a dictionary of the keyword arguments for the profile function. (It can also include unused keyword arguments.)

In [ ]:
def wake_sig(k, y, z, cntr_pos=[0, 90], rad=50, sig_inf=1.2, max_perc=0.20, **kwargs):
    """Non-realistic wake turbulence. sig_inf is the undisturbed standard deviation and max_perc is the
    maximum percentage increase of sigma at the center."""
    dist_to_cntr = np.sqrt((y - cntr_pos[0])**2 + (z - cntr_pos[1])**2)  # distance to center point
    mask = (dist_to_cntr < rad)  # points that fall within the wake
    wake_sig = sig_inf * constant_profile(y, z, u_ref=1)  # freestream sig
    wake_sig[mask] += max_perc*sig_inf * np.sin(np.pi/2 * (rad - dist_to_cntr[mask]) / rad)
    return np.array(wake_sig)  # must return array regardless of input

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10, 3.7))
plot_vals = [([-60, 160], 1.2, 0.2), ([10, 100], 3.5, 0.4)]
for iax, (cntr, sig, mperc) in enumerate(plot_vals):
    ax = axs[iax]
    sig_values = wake_sig(spat_df.loc['k'], spat_df.loc['y'], spat_df.loc['z'],
                          cntr_pos=cntr, rad=rad, sig_inf=sig, max_perc=mperc)
    cnt = ax.contourf(y, z, sig_values.reshape(y.size, z.size).T)
    ax.axis('equal')
    plt.colorbar(cnt, ax=ax);
    ax.set_title(f'Max +{mperc*100}% at {cntr}, sig_inf={sig}')

## Generate and inspect turbulence

Now that we've created (and verified) our custom wind speed and turbulence standard deviation functions, we can run our turbulence generation. We first put all of our simulation options into a keyword argument dictionary and then pass those into `gen_turb`. Note that `u_ref` is no longer used in the wind speed or sig profiles, but it is needed for the Kaimal spectrum function.

In [ ]:
kwargs = {'T': T, 'dt': dt, 'u_ref': 10, 'wsp_func': wake_wsp, 'sig_func': wake_sig,
          'cntr_pos': cntr_pos, 'rad': rad, 'u_inf': u_inf, 'sig_inf': 1, 'max_perc': 2}
turb_df = gen_turb(spat_df, **kwargs)

We can inspect the resulting turbulence block to see if it matches our expectations.

**Size/elements:**

In [ ]:
turb_df.head()

**Mean wind speed profile:**

In [ ]:
mean_wsp = turb_df.mean().values
print('Min mwsp: ', mean_wsp.min(), '  Max mwsp: ', mean_wsp.max())

plt.imshow(np.reshape(mean_wsp, (y.size, z.size)).T, interpolation='none',
           origin='lower', extent=[y.min(), y.max(), z.min(), z.max()])
plt.colorbar();

This looks exactly as we expect it to: the mean wind speed outside the waked area is 12 m/s, and the maximum deficit is 0.5 m/s. Excellent.

**Turbulence standard deviation:**

In [ ]:
std_wsp = turb_df.std().values
print('Min std: ', std_wsp.min(), '  Max std: ', std_wsp.max())

plt.imshow(np.reshape(std_wsp, (y.size, z.size)).T, interpolation='none',
           origin='lower', extent=[y.min(), y.max(), z.min(), z.max()])
plt.colorbar();

As you can see, there is more variation in the turbulence profile. This is due to the spatial correlation procedure that occurs during the Veers method simulation; we will never get the exact TI profile we specify. However, we still generally get the correct values: 1.0 m/s in the freestream and 3.0 (200% higher) at the highest.